# Quantum Buffon's Needle Problem Demonstration

## Abstract

Quantum computes promise numerous advantages, including quantum parallelism, quantum simulation, and quantum cryptograhpy.
Since the theory of local hidden variables has been disproven, it is established that quantum systems are capable of generating truly random outputs.
By preparing qubits in superposition states, one can obtain uniformly distributed random outputs.
This notebook presents a demonstration of quantum randomness through Buffon's needle problem, a probabilistic mehtod for estimating the value of $\pi$.
In this problem, needles of fixed length are randomly dropped onto a plane with evenly spaced parallel lines, and the ratio between the number of intersections and the total number of throws converges to $\pi$.
The random positions and orientations of the needles are generated using a quantum computer, effectively simulating the experiment.
The purpose of this work is not to accelerate the computation of $\pi$, but rather to illustrate the quality of randomness obtainable from quantum processes and to highlight their potential in probabilistic simulations.
This demonstration connects a mathematical conclusion to modern quantum technology, offering both pedagogical value and an example of the interplay between probability and quantum computation.

## Introduction

### Buffon's Needle Problem

We shall begin by introducing the context behind Buffon's needle problem.
Consider a plane marked with evenly spaced horizontal parallel lines, separated by distance $D$.
A set of needles, each of length fixed $l < D$, is randomly dropped onto the plane.
Some needles will intersect (or cross) one of the lines, while others will land entirely in the blank spaces between the lines.
If we let $N_{\text{cross}}$ denote the number of needles that intersect a line and $N_{\text{total}}$ the total number of needles thrown, then the ratio $\frac{N_{\text{cross}}}{N_{\text{total}}}$ converges, as the number of throws increases, to the true probability $\mathbb{P}_{\text{cross}}$ of a needle crossing a line.
Consequently, the probability can be approximated by:
$$\mathbb{P}_{\text{cross}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} \text{.}$$

Let us simplify the situation.
We can remove all the lines on the plane except for two consecutive ones separated by distance $D$.
Now place a needle between these lines with a random orientation $\theta$.

We can look at this problem from a different perspective.
Instead of focusing on the two lines, imagine throwing a line segment of length $D$ and checking whether it lands on top of the needle.
In this setting, the position (both horizontal and vertical) position of the needle does not matter; only the vertical projection of the needle, denoted $y$, is relevant.
By symmetry, it suffices to restrict the orientation to the range $0 \leq \theta \leq \frac{\pi}{2}$.

Once a line is thrown into the strip, the probability that it lands on the needle is $\frac{y}{D}$.
Since the vertical projection can be expressed in terms of the orientation, $y = l\sin(\theta)$, we have:
$$P(\theta) = \frac{y}{D} = \frac{l\sin(\theta)}{D} \text{.}$$

Here, $P(\theta)$ represents the conditional probability of a crossing given that the needle has orientation $\theta$.
However, in the actual experiment, the orientation $\theta$ itself is random and uniformly distributed in $\left[0, \frac{\pi}{2}\right]$.
To obtain the overall crossing probability, we therefore compute the expected value of $P(\theta)$ over all possible orientations in $\left[0, \frac{\pi}{2}\right]$.
$$
\begin{align*}
    \mathbb{P}_{\text{cross}} &= \mathbb{E}[P(\theta)] \\
    &= \frac{1}{\frac{\pi}{2}-0} \int_{0}^{\frac{\pi}{2}} P(\theta) \,\text{d}\theta \\
    &= \frac{2}{\pi} \int_{0}^{\frac{\pi}{2}} \frac{l\sin(\theta)}{D} \,\text{d}\theta \\
    &= \frac{2l}{\pi{D}} \left[-\cos(\theta)\right]_{\theta=0}^{\theta=\frac{\pi}{2}} \\
    &= \frac{2l}{\pi{D}}
\end{align*}
$$

From the theoratical probability we just derived, we can now construct a formula to approximate the value of $\pi$.
Let us denote the approximated $\pi$ value as $\hat{\pi}$.
$$
\begin{align*}
    \mathbb{P}_{\text{cross}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} &\implies \frac{2l}{\pi{D}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} \\
    &\implies \frac{1}{\pi} \approx \frac{DN_{\text{cross}}}{2lN_{\text{total}}} \\
    &\implies \pi \approx \frac{2lN_{\text{all}}}{DN_{\text{cross}}} \\
    &\implies \boxed{\hat{\pi} \equiv \frac{2lN_{\text{all}}}{DN_{\text{cross}}}}
\end{align*}
$$

To simulate this problem, we need to randomly generate two floating-point numbers: the vertical position of the thrown line, $y' \in [0, D]$, and the orientation of the needle, $\theta \in \left[0, \frac{\pi}{2}\right]$.
A crossing is detected by cehcking whether $y' \leq y = l\sin(\theta)$.

Next, we shall describe how to generate uniformly random bits using a quantum computer and how to convert the retrieved bit strings into uniformly distributed floating-point numbers within a given range.

### Random Floating-Point Number Generation

6. Double floting point datatype and mantissa.
7. Normalization and scaling.

Classical mechanics promised that every phenomenon could be predicted exactly if the initial conditions of the system were known with high precision.
However, the advent of quantum mechanics challenged this deterministic view.
According to the Copenhagen interpretation, quantum systems are fundamentally probabilistic; we can derive probabilities of outcomes with stochastic reasoning, but the actual result of a measurement cannot be determined in advance.
This led some to speculate that there might exist hidden variables, unknown parameters that would restore determinism.
The local hidden variables theory was proposed under the idea that under the assumption of the hidden variables were known, the outcome of any quantum measurement could be predicted deterministically.

Later, John Stewart Bell formalized this question and proposed an idea.
He showed that any local hidden variable thoery must satisfy certain constraints, known as Bell inequalities.
However, continued experiments consistently showed that Bell inequalities do not hold, confirming that local hidden variables cannot fully explain quantum measurement outcome.
Since no alternative deterministic thoery has been proposed, quantum randomness is considered genuine and fundamental, not merely a reflection of incomplete knowledge.
Quantum computers leverages these quantum phenomena to encode and process information, enabling the generation of truly random numbers.
This is fundamentally different from pseudo-random number generators on classical computers, which are deterministic and can be reproduced if the initial seed is known.

In most quantum computing models, each qubit (quantum bit) is initially prepared in the standard state $\ket{0}$.
As the computation progresses over time, qubits are manipulated through a sequence of quantum gates, which create superpositions and enable interferences between possible computational paths.
These operatoins allow the quantum system to explore multiple solutions simultaneously.
At the end of the computation, each qubit is measured, collapsing its quantum state to a classical bit (exclusively 0 or 1).
The collection of measured bits forms a classical output that reflects the probabilistic nature of the underlying quantum computation.

A quantum gate that puts a qubit into a superposition state is the Hadamard gate ($H$).
In matrix form, the Hadamard gate is defined as:
$$H = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}\text{.}$$
The standard basis states $\ket{0}$ and $\ket{1}$ are represented as:
$$
\begin{align*}
    &\ket{0} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}\text{,}
    &\ket{1} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}\text{.}
\end{align*}
$$
The resulting state after applying the Hadamard gate can be computed by multiplying the matrix with the basis vector:
$$
\begin{align*}
    H\ket{0} &= \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \end{bmatrix} \\
    &= \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix} \\
    &= \frac{1}{\sqrt{2}}\ket{0} + \frac{1}{\sqrt{2}}\ket{1}\text{.}
\end{align*}
$$
The coefficient in front of the standard basis states are called probability amplitudes.
The probability of observing a particular basis state is given by the squared modulus of its amplitude.
Thus, after passing the initial state $\ket{0}$ through the Hadamard gate, there is a $\left|\frac{1}{\sqrt{2}}\right|^2 = \frac{1}{2} = 50\%$ chance in measuring 0 and a $\left|\frac{1}{\sqrt{2}}\right|^2 = \frac{1}{2} = 50\%$ chance in measuring $1$.
This state can generate one bit long uniformly random binary string.

Now, let us expand the discussion from the single-qubit case to an arbitrary number of qubits.
To describe the state of multiple qubits stimultaneously, we use the tensor products of individual qubit states, denoted by $\otimes$.
For example, in a two-qubit system, we can represent the combined state by taking the tensor product of the individual qubits.
$$
\begin{align*}
    \ket{0} \otimes \ket{0} &= \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} \\
    &= \begin{bmatrix} 1\begin{bmatrix} 1 \\ 0 \end{bmatrix} \\ 0\begin{bmatrix} 1 \\ 0 \end{bmatrix} \end{bmatrix} \\
    &= \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix} \\
    &= \ket{00} = \ket{0^{\otimes 2}} \\
    \ket{0} \otimes \ket{1} &= \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} \\
    &= \begin{bmatrix} 1\begin{bmatrix} 0 \\ 1 \end{bmatrix} \\ 0\begin{bmatrix} 0 \\ 1 \end{bmatrix} \end{bmatrix} \\
    &= \begin{bmatrix} 0 \\ 1 \\ 0 \\ 0 \end{bmatrix} \\
    &= \ket{01} \\
    \ket{1} \otimes \ket{0} &= \begin{bmatrix} 0 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} \\
    &= \begin{bmatrix} 0\begin{bmatrix} 1 \\ 0 \end{bmatrix} \\ 1\begin{bmatrix} 1 \\ 0 \end{bmatrix} \end{bmatrix} \\
    &= \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix} \\
    &= \ket{10} \\
    \ket{1} \otimes \ket{1} &= \begin{bmatrix} 0 \\ 1 \end{bmatrix} \otimes \begin{bmatrix} 0 \\ 1 \end{bmatrix} \\
    &= \begin{bmatrix} 0\begin{bmatrix} 0 \\ 1 \end{bmatrix} \\ 1\begin{bmatrix} 0 \\ 1 \end{bmatrix} \end{bmatrix} \\
    &= \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix} \\
    &= \ket{11} = \ket{1^{\otimes 2}}
\end{align*}
$$
Similarly, quantum gates acting on multiple qubits can be expressed using tensor products.
$$
\begin{align*}
    H \otimes H &= \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \otimes \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \\
    &= \frac{1}{2} \begin{bmatrix} 1\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} & 1\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \\ 1\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} & -1\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \end{bmatrix} \\
    &= \frac{1}{2} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{bmatrix} \\
    &= H^{\otimes 2}
\end{align*}
$$

Let us denote by $\ket{\psi_n}$ the state obtained after applying Hadamard gates to $n$ qubits.
We can now make the claim $P(n)$: $\ket{\psi_n}$ can generate a uniformly random $n$-bit binary string; this claim can be proven by inductive reasoning.
Assume $P(k)$ holds for an arbitrary $k \in \mathbb{N}$, meaning that $\ket{\psi_k} = H^{\otimes k}\ket{0^{\otimes k}}$ generates a uniformly random $k$-bit binary string.
Now, append another qubit initialized in $\ket{0}$ and apply a hadamard gate to it.
The resulting $(k+1)$-qubit state is:
$$
\begin{align*}
    \ket{\psi_{k+1}} &= H^{\otimes k}\ket{0^{\otimes k}} \otimes H\ket{0} \\
    &= H^{\otimes k}\ket{0^{\otimes k}} \otimes \left(\frac{1}{\sqrt{2}}\ket{0} + \frac{1}{\sqrt{2}}\ket{1}\right) \\
    &= \frac{1}{\sqrt{2}} H^{\otimes k}\ket{0^{\otimes k}} \otimes \ket{0} + \frac{1}{\sqrt{2}} H^{\otimes k}\ket{0^{\otimes k}} \otimes \ket{1}\text{.}
\end{align*}
$$
Since the new qubit has an equal probability of being measured as 0 or 1, the state $\ket{\psi_{k=1}}$ generates a uniformly random $(k+1)$-bit binary string.
By mathematical induction, we conclude that any number $n$ of qubits prepared with parallel Hadamard gates can generate uniformly random $n$-bit binary strings.

To convert an $n$-bit binary string into a floating-point number within a specified range $[a, b]$, we first interpret the string as an integer $I$ between $0$ and $2^n-1$.
Supposed we put the $n$-bit binary string $b = b_{n-1}b_{n-2} \cdots b_0$, where $b_i \in \set{0, 1}$.
To interpret it as an integer $I$, we compute the following:
$$I = \sum_{i=0}^{n-1} b_i \cdot 2^i\text{.}$$
By dividing this integer by $2^n-1$, we obtain a normalized value $x = \frac{I}{2^n-1}$ such that $x \in [0, 1]$.
This normalized value can then be scaled to any arbitrary interval $[a, b]$ using the formula $y = a + (b-a)x$, ensuring that the binary string maps uniquely to a number in the desired range.
When representing these numbers in IEEE 754 double-precision floating-point format, it is imporant to note that the mantissa contains 52 stored bits, but due to the implicit leading 1, the effective precision is 53 bits.
Thus, any binary string of length up to 53 bits can be represented exactly, while longer strings may lose some precision due to rounding.

## Demonstration

### Installation

Assuming that Python is already installed on the host device, the next step is to install the essential packages required for this demonstration, beginning with Qiskit.
Qiskit (**Q**uantum **I**nformation **S**cience **Kit**) is an open-source framework maintained by IBM.
With Qiskit, we can compose quantum circuits, simulate their behavior, and execute them on real quantum hardware.

Two related packages are particularly important for this demonstration:
* `qiskit-aer`, which provices high-performance quantum circuit siulators, and
* `qiskit-ibm-runtme`, which allows circuits to be run on actual quantum devices manufactured by IBM via IBM cloud.

In addition to Qiskit, we will also use two supporting packages:
* `matplotlib`, for generating plots for a comparison, and
* `tqdm`, for displaying progress bars when iterating over `for` loops in Python.

Since it is known that Python's `for` loops are relatively slower than other languages, `tqdm` helps by clearly tracking progress and providing an estimated completion time.

All of these packages can be installed (or updated if present) with a single command as the following:

In [ ]:
%pip install -U qiskit qiskit-aer qiskit-ibm-runtime matplotlib tqdm

Now, we can import those packages into our Python environment with the following code:

In [ ]:
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_aer.primitives import SamplerV2 as Simulator
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

### Classical Simulation

Generally, IBM provides free-tier users with 10 minutes of computation per month on their quantum hardware.
This allocation is quite generous, ocnsidering the immmerse powe required to maintain quantum states within these devices.
However, jobs submitted to real quantum hardware often face long waiting times, and it is not encouraged to spend limited quantum runtime on debugging or testing our quantum circuits.
A more practical approach is to first test and validate quantum circuits and algorithms on classical simulators at a smaller scale before sending them to actual quantum devices.

Let us define a few helper functions in Python that we will ues throughout the demonstration.
We begin with the function `get_qc`.
This function takes as the number of qubit (`num_qubits`) and returns a `QuantumCircuit` where each qubit is placed into superposition using a Hadamard gate ($H$), followed by a measurement operation on every qubit at the end.

In [ ]:
def get_qc(num_qubits: int) -> QuantumCircuit:
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        qc.h(i)
    qc.measure_all()
    
    return qc

We now move on to the function `get_counts_simulation`.
This function takes a quantum circuit `qc` and the number of needle throws `throws`.
Once these parameters are provided, the function executes the circuit simulation and collects the measurement results as `counts`.

Because each throw of a needle requires to independent random numbers (the position $y'$ and the orientation $\theta$), as described above, the circuit is executed twice the number of throws.
Here, the `shots` argument of the `run` function is set to be `2 * throws`.
The tasks of normalizing and scaling these raw random numbers will be handled later.

The returned variable `counts` is a Python dictionary whose keys are the measured bit strings and whose values are the corresponding frequencies with which those bit strings were observed.
As we increase the number of qubits, it becomes more unlikely to measure the identical bit strings twice, but we must take of such cases.

In [ ]:
def get_counts_simulation(qc: QuantumCircuit, throws: int) -> dict:
    simulator = Simulator()
    job = simulator.run([qc], shots=2*throws)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    return counts

The next function is `get_binary_string`.
This function takes `counts` as input and selects a binary string.
It also manages the number of times used: if the frequency of the chosen binary string is greater than one, the count is decremented by one; if the frequency is exactly one, the entry is removed from the `counts` dictionary.

In [ ]:
def get_binary_string(counts: dict) -> str:
    picked = list(counts.keys())[0]

    if counts[picked] == 1:
        del counts[picked]
    else:
        counts[picked] -= 1

    return picked

We can convert a randomly generated binary string into a floating-point number within a specified range using the function `get_float`.
The methodology of this function was described in the previous section.

In [ ]:
def get_float(string: str, start: float, end: float) -> float:
    integer = int(string, 2)
    normalized = integer / (2 ** len(string) - 1)
    scale = end - start

    return start + scale * normalized

The final helper function is `throw_needle`, which simulates the process of tossing a neelds in Buffon's needle problem and returns whether the needle crosses a line.
As described in the previous section, the crossing is detected by checking whether $y' \leq l\sin(\theta)$.

In [ ]:
def throw_needle(needle_length: float, position: float, orientation: float) -> bool:
    return position <= needle_length * math.sin(orientation)

Here are the main parameters:
* `strip_length`, representing the distance $D$ between the lines,
* `needle_length`, representing the needle length $l$,
* `num_qubits`, specifying the number of qubits (i.e., the length of each binary string), and
* `throws`,specifying the number of needle throws.

Readers are encouraged to adjust these preset parameters, provided they satisfy the predefined condition $l < D$.

In [ ]:
strip_length = 5.0 # Your strip length here.
needle_length = 3.0 # Your needle length here.
num_qubits = 53 # Your number of qubits here.
throws = 50_000 # Your number of needle throws here (not recommended to exceed 50,000).

if not needle_length < strip_length:
    raise ValueError("Needle length must be less than strip length.")

Now, we run the actual algorithm.
For each toss, the number of corssings `cross` and the number of thrown needles at that moment `total` are tracked.
If `cross` is greater than zero, the approximated value $\hat{\pi}$ is saved in the `approx_pi` variable, and its error is computed by $\frac{|\hat{\pi}-\pi|}{\pi}$.
After the for loop, the program plots the number of throws vsrsus the error on a log-log scale, which cearly shows the trend.

In [ ]:
qc = get_qc(num_qubits)
counts = get_counts_simulation(qc, throws)
width = len(str(throws))

cross, total = 0, 0
throw_list_simulation, error_list_simulation = [], []
report_string = ""

for i in tqdm(range(1, throws+1)):
    position = get_float(get_binary_string(counts), 0.0, strip_length)
    orientation = get_float(get_binary_string(counts), 0.0, math.pi/2.0)

    cross += 1 if throw_needle(needle_length, position, orientation) else 0
    total += 1

    if cross > 0:
        approx_pi = (2.0 * needle_length * total) / (strip_length * cross)
        error = abs(approx_pi - math.pi) / math.pi
        error_perc = error * 100.0

        throw_list_simulation.append(i)
        error_list_simulation.append(error)

    if i % (throws // 20) == 0:
        report_string += f"Throw {i:{width}}: π ≈ {approx_pi:.10f} (error rate: {error_perc:.10f}%)\n"

print(report_string)

plt.figure()
plt.loglog(throw_list_simulation, error_list_simulation, linestyle='-', color='blue')
plt.xlabel("Number of Throws")
plt.ylabel("Error")
plt.title("Convergence of Buffon's Needle Simulation (Simulation)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### Quantum Implementation

Moving on to the implementation on actual quantum hardware, we must define a separate function to retrieve the counts dictionary: `get_counts_quantum`.
Its parameters are idential to the previous function, with the addition of `token` and `crn`, which are required for accessing IBM cloud services.
After executing the quantum circuit on the hardware, the function returns a dictionary in the same format as before, mapping bit strings to their corressponding measured frequencies.

In [ ]:
def get_counts_quantum(qc: QuantumCircuit, throws: int, token: str, crn: str) -> dict:
    service = QiskitRuntimeService(channel='ibm_quantum_platform', token=token, instance=crn)
    backend = service.backends(simulator=False, operational=True)[0]

    pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
    trans_qc = pm.run(qc)
    
    sampler = Sampler(mode=backend)
    job = sampler.run([trans_qc], shots=2*throws)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    return counts

Readers need to provide their own token and CRN from the IBM Cloud website in order to proceed.

In [ ]:
token = "" # Your IBM API token here.
crn = "" # Your IBM CRN here.

Similar to the classical simulation, this will plot the number of throws versus the error on a log-log scale.

In [ ]:
qc = get_qc(num_qubits)
counts = get_counts_quantum(qc, throws, token, crn)
width = len(str(throws))

cross, total = 0, 0
throw_list_quantum, error_list_quantum = [], []
report_string = ""

for i in tqdm(range(1, throws+1)):
    position = get_float(get_binary_string(counts), 0.0, strip_length)
    orientation = get_float(get_binary_string(counts), 0.0, math.pi/2.0)

    cross += 1 if throw_needle(needle_length, position, orientation) else 0
    total += 1

    if cross > 0:
        approx_pi = (2.0 * needle_length * total) / (strip_length * cross)
        error = abs(approx_pi - math.pi) / math.pi
        error_perc = error * 100.0

        throw_list_quantum.append(i)
        error_list_quantum.append(error)

    if i % (throws // 20) == 0:
        report_string += f"Throw {i:{width}}: π ≈ {approx_pi:.10f} (error rate: {error_perc:.10f}%)\n"

print(report_string)

plt.figure()
plt.loglog(throw_list_quantum, error_list_quantum, linestyle='-', color='red')
plt.xlabel("Number of Throws")
plt.ylabel("Error")
plt.title("Convergence of Buffon's Needle Simulation (Quantum)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Discussion

| Classical Result               | Quantum Result                 |
|:------------------------------:|:------------------------------:|
| ![Plot 1](./figures/plot1.png) | ![Plot 2](./figures/plot2.png) |

In both the classical and quantum simulations, the algorithm successfully approximated the value of $\pi$.
As the number of needle throws increased, the estimated value of $\pi$ approached the true value, although some fluctuations were observed.
These fluctuations are a natural consequence of the randomness inherent in the sampling process, whether it originates from classical pseudo-random number generation or quantum randomness.

This demonstration highlights the effectiveness of quantum randomness as a reliable source of truly random numbers.
Quantum computers are capable of generating random binary strings, integers, and floating-point numbers, which can then be used in probabilistic simulations and computations.
In this experiment, we used the random floating-point numbers generated by quantum hardware to simulate the positions ($y'$) and orientations ($\theta$) of the needles.
The results show that even with a limited number of qubits and measurements, quantum-generated randomness is sufficient for modeling probabilistic systems, including Buffon's needle problem.

Despite these successes, there are some limitations when using quantum hardware for this type of algorithm.
Real quantum devices are subject to errors caused by decoherence, gate imperfections, and readout noise,which can slighty distort the distributino of generated random numbers.
Additionally, the convergence to high precision approximation of $\pi$ is relatively slow.
Therefore, while this demonstration is valueable for illustrating quantum randomness and its applications, it is not intended as a practical method for high precision computation of $\pi$.

## Conclusion


Overall, this demonstration bridges a classic probabilistic problem with modern quantum technology, providing an educational example of how quantum computers can be used to generate high-quality randomness for simulations, probabilistic algorithms, and even password generation.

## References

* Bernhardt, C. *Quantum Computing for Everyone*. MIT Press, 2019.
* “Buffon’s Needle Problem,” *MathWorld — A Wolfram Web Resource*. Available at: https://mathworld.wolfram.com/BuffonsNeedleProblem.html
* “Introduction of Floating Point Representation,” *GeeksforGeeks*. Available at: https://www.geeksforgeeks.org/digital-logic/introduction-of-floating-point-representation/#
* IEEE Standard for Binary Floating-Point Arithmetic (IEEE 754-1985). Available at: https://www.ime.unicamp.br/~biloti/download/ieee_754-1985.pdf